## Planner

Semantic Kernel bringt auch das Feature Planner mit.
Die Idee hinter Planners ist es, dass diese GPT nutzen um herauszufinden welche Plugins in welcher reihenfolge aufgerufen werden sollen.

#### Erstmal wollen wir wieder all unsere Plugins laden

In [6]:
// setup
#!import config/Settings.cs
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Planners;
using System.ComponentModel;

//Create Kernel builder
var builder = new KernelBuilder();
// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta1

In [5]:
#!import plugins/DevOpenspacePlugin/DevOpenSpacePlugin.cs
using Microsoft.SemanticKernel.Plugins.Core;

var devopenspaceplugin = kernel.ImportFunctions(new DevOpenSpacePlugin(), "DevOpenspacePlugin");
var conversationSummaryPlugin = kernel.ImportFunctions(new ConversationSummaryPlugin(kernel), "ConversationSummaryPlugin");

var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "plugins");
var unterhaltungsPluginFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "UnterhaltungsPlugin");


Und nun nehmen wir uns den built-in SequentialPlanner und lassen ihn mal "Plannen"

In [9]:
var planner = new SequentialPlanner(kernel);

var userask = "Mecker";

var plan = await planner.CreatePlanAsync(userask);

var result = await plan.InvokeAsync();

Error: Microsoft.SemanticKernel.Diagnostics.SKException: Not possible to create plan for goal with available functions.
Goal:Mecker
Functions:

   at Microsoft.SemanticKernel.Planners.SequentialPlanner.CreatePlanAsync(String goal, CancellationToken cancellationToken)
   at Submission#19.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:

var result = await kernel.RunAsync("", devopenspaceplugin["GetUsersTwitterHandle"], devopenspaceplugin["GetUsersCount"], unterhaltungsPluginFunctions["Mecker"]);

// Antwort ausgeben
Console.WriteLine(result.GetValue<string>());